In [1]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import (
    show_cam_on_image, deprocess_image, preprocess_image
)
from torchvision.models import resnet18
import torchvision.transforms as transforms
import torch
import cv2
import numpy as np

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = resnet18(num_classes=2).to(device).eval()
model.load_state_dict(torch.load('explicit_label.pkl'))
rgb_img = cv2.imread('./noiseImageLabel1.png', 1)[:, :, ::-1]
rgb_img = np.float32(rgb_img) / 255
input_tensor = preprocess_image(rgb_img,
                                mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225]).to(device)
print('model prediction vector is:', model(input_tensor))
print('predict result',model(input_tensor).argmax(1))

model prediction vector is: tensor([[-6.3110,  5.6962]], device='cuda:0', grad_fn=<AddmmBackward0>)
predict result tensor([1], device='cuda:0')


In [20]:
target_layers = [model.layer4[-1]]

In [21]:
def show_cam_on_image(img: np.ndarray,
                      mask: np.ndarray,
                      use_rgb: bool = False,
                      colormap: int = cv2.COLORMAP_JET,
                      image_weight: float = 0.5) -> np.ndarray:
    
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), colormap)
    if use_rgb:
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    heatmap = np.float32(heatmap) / 255

    if np.max(img) > 1:
        raise Exception(
            "The input image should np.float32 in the range [0, 1]")

    if image_weight < 0 or image_weight > 1:
        raise Exception(
            f"image_weight should be in the range [0, 1].\
                Got: {image_weight}")

    cam = (1 - image_weight) * heatmap + image_weight * img
    cam = cam / np.max(cam)
    return np.uint8(255 * cam)

In [22]:
# CAM object
targets = None
cam_algorithm = GradCAM
with cam_algorithm(model=model,
                       target_layers=target_layers) as cam:
    cam.batch_size = 32
    # Target is for downstream tasks and 281 is for specific class?
    
    # Why grayscale_cam?
    grayscale_cam = cam(input_tensor=input_tensor, 
                        targets=targets,
                        )
    # ??
    grayscale_cam = grayscale_cam[0, :]
    visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
    print(visualization)
    visualization = cv2.cvtColor(visualization, cv2.COLOR_RGB2BGR)

[[[127 124   0]
  [127 124   0]
  [127 124   0]
  ...
  [ 77 127  51]
  [ 77 127  51]
  [ 77 127  51]]

 [[127 124   0]
  [127 124   0]
  [127 124   0]
  ...
  [ 77 127  51]
  [ 77 127  51]
  [ 77 127  51]]

 [[127 124   0]
  [127 124   0]
  [127 124   0]
  ...
  [ 77 127  51]
  [ 77 127  51]
  [ 77 127  51]]

 ...

 [[127 116   0]
  [127 116   0]
  [127 116   0]
  ...
  [ 29 127  99]
  [ 29 127  99]
  [ 29 127  99]]

 [[127 116   0]
  [127 116   0]
  [127 116   0]
  ...
  [ 29 127  99]
  [ 29 127  99]
  [ 29 127  99]]

 [[127 116   0]
  [127 116   0]
  [127 116   0]
  ...
  [ 29 127  99]
  [ 29 127  99]
  [ 29 127  99]]]


In [23]:
cv2.imwrite('./interpretation_GradCam/0.jpg',visualization)

True